Assignment 2 of Hanson
Legal Name Hanxiang Zhang
Stu# 300100018

1 Normalized Face Image Comparison
1.1 I wrote a function named data_preprocessing to read and process the image data.
I downloaded the two subsets and unpacked the images in sub1to10 and sub31to40 directories relative to my jupyter notebook working path.
To load all images, I generated a list of paths to the image files by reading the directory and file structure using the os package in python.
I shuffled the lists of paths of training & validation images and testing images respectively to randomlize the data.
Then I read the images, converted them to grayscale and rescaled them to 64 x 64.
Finally, I used if-else statements to assign all the images with its labels, which is the number shown in its folder indicating which person it is.

There are 460 images for training and validation, in sub1to10.
There are 384 images for testing, in sub31to40.

In [1]:
import os
import numpy as np
import random
import skimage
from skimage import io
from skimage import transform

def data_preprocessing():
    #Get the current working directory
    dire = os.getcwd()

    path_train = dire+'\\sub1to10\\'
    path_test = dire+'\\sub31to40\\'

    #Read images for training and validation from sub1to10
    train_files = []
    for root, dirs, files in os.walk(path_train):
        for file in files:
            if '.png' in file:
                train_files.append(os.path.join(root, file))

    #Read images for testing from sub31to40
    test_files = []
    for root, dirs, files in os.walk(path_test):
        for file in files:
            if '.png' in file:
                test_files.append(os.path.join(root, file))

    #Randomly Shuffle the training image files and testing image files respectively
    random.shuffle(train_files)
    random.shuffle(test_files)

    #print(len(train_files))
    #print(len(test_files))

    #Read and processing image data, and assign their corresponding labels
    train_images = []
    train_labels = []
    test_images = []
    test_labels = []

    #For training and validation data
    #Read every images, convert them into grayscale images, and rescale them to 64 by 64
    #Then add each image for training into train_images list,
    #together with its label into train_labels list;
    #And add each image for testing into test_images list,
    #together with its label into test_labels list.
    for (i, imgpath) in enumerate(train_files):#training files
        img = skimage.io.imread(imgpath)
        imgg = skimage.color.rgb2gray(img)
        imgu = skimage.transform.resize(imgg, (64,64)) #resize image to 64x64
        #if i==1:
        #   print(imgu)
        train_images.append(imgu)
        lb=-1
        if "sub1_" in imgpath:
            lb=1
        elif "sub2_" in imgpath:
            lb=2
        elif "sub3_" in imgpath:
            lb=3
        elif "sub4_" in imgpath:
            lb=4
        elif "sub5_" in imgpath:
            lb=5
        elif "sub6_" in imgpath:
            lb=6
        elif "sub7_" in imgpath:
            lb=7
        elif "sub8_" in imgpath:
            lb=8
        elif "sub9_" in imgpath:
            lb=9
        elif "sub10_" in imgpath:
            lb=10
        else:
            lb=-1
            print("Something wrong going on when labelling training data")
        train_labels.append(lb)

    #For testing data, the same as above
    for (i, imgpath) in enumerate(test_files):
        img = skimage.io.imread(imgpath)
        imgg = skimage.color.rgb2gray(img)
        imgu = skimage.transform.resize(imgg, (64,64))
        test_images.append(imgu)
        lb=-1
        if "sub31_" in imgpath:
            lb=31
        elif "sub32_" in imgpath:
            lb=32
        elif "sub33_" in imgpath:
            lb=33
        elif "sub34_" in imgpath:
            lb=34
        elif "sub35_" in imgpath:
            lb=35
        elif "sub36_" in imgpath:
            lb=36
        elif "sub37_" in imgpath:
            lb=37
        elif "sub38_" in imgpath:
            lb=38
        elif "sub39_" in imgpath:
            lb=39
        elif "sub40_" in imgpath:
            lb=40
        else:
            lb=-1
            print("Something wrong going on when labelling testing data")
        test_labels.append(lb)
    print("Image preprocessing finished.")
    #print(train_images)
    #print(test_images)
    #print(train_labels)
    #print(len(train_labels))
    #print(len(test_labels))
    return train_images, train_labels, test_images, test_labels

#Run the function
train_images, train_labels, test_images, test_labels=data_preprocessing()

Image preprocessing finished.


1.2	Template Matching [2.5]
Write a function that accepts two of the images and calculates a score based on the similarity of the two images using either cross-correlation, convolution or sum of squared differences. Choose your method carefully and perform any processing step necessary. Given the similarity score, perform a simple classification using a threshold to decide if two images show the same face. Evaluate your classifier with pairs chosen from the 460 images in the training data set. There are a total of  111 pairs but most of the pairs will show different faces. A small validation set of 1,000 pairs of images seems therefore reasonable.

1.2 Template Matching
I wrote a function named generate_pairs to generate image pairs and template_matching for this task of template matching.

First of all, this accept images and generate pairs of 2 non-duplicate images. The pairs are generated by using a nested loop with i_list and j_list. The indeces of pairs are stored in i_list and j_list. The index stored in i_list corresponds to the index in j_list in the same position. For example, for the pair (0,1), 0 is in the first postion in i_list and 1 is in the first position in j_list. The pairs generated are still random even though I make the index in order because I have shuffled the dataset beforehead when I get the image path list.

The number of pairs for training and validation should be 105570=(460*459)/2 in total. And the number of pairs for training is 104570, while the number of pairs for validation is 1000. I made the last 1000 image pairs as the validation. This is ok since data has been shuffled previously.

The number of pairs for testing is 73536=(384*383)/2.

In [2]:
import scipy as sp
from skimage.feature import match_template
import numpy as np

def generate_pairs():
    #For training and validation image pairs,
    #Generate the pairs of 2 images from the 460 images.
    i_list=[]
    j_list=[]
    counter=0
    #Generate the indices of image pairs using nested loop.
    #I shuffled the dataset before, so I can make the numbers in order
    for i in range(0,460):
        for j in range(i+1,460):
            i_list.append(i)
            j_list.append(j)
            counter=counter+1
    print(counter)
    print(len(i_list))
    print(len(j_list))

    #Divide the training and validation dataset.
    #I shuffled the dataset before, so I can simply use the last 1000 pairs as validation pairs
    train_i_list=i_list[:104570]
    train_j_list=j_list[:104570]

    valid_i_list=i_list[104570:]
    valid_j_list=j_list[104570:]
    print(len(train_i_list))
    print(len(train_j_list))
    print(len(valid_i_list))
    print(len(valid_j_list))

    
    #For testing image pairs,
    #Generate the pairs of 2 images from the 384 images.
    test_i_list=[]
    test_j_list=[]
    counter_t=0
    #I shuffled the dataset before, so I can make the numbers in order
    for i in range(0,384):
        for j in range(i+1,384):
            test_i_list.append(i)
            test_j_list.append(j)
            counter_t=counter_t+1
    print("Image pair generation Finished")
    return train_i_list, train_j_list, valid_i_list, valid_j_list, test_i_list, test_j_list
    
train_i_list, train_j_list, valid_i_list, valid_j_list, test_i_list, test_j_list=generate_pairs()

105570
105570
105570
104570
104570
1000
1000
Image pair generation Finished


After the indices of pairs of images are generated, I calculate the similarity score using cross-correlation. This is achieved by using match_template in skimage. It uses a Fast Normalized Cross-Correlation approach by J. P. Lewis which employs tables that have integrals of two images that are obtained previously for normalization with reduced computing time [1].
The score ranges from -1 to 1, which is correlation coefficient. while here most of the them are from 0 to 1. A higher score indicates a higher similarity.

I tried the threshold values of 0.7, 0.75, 0.8, 0.85, 0.9 and got the results shown below. The images being classified as the same person is the positive class. The accuracy seem high for all the threshold values, but this is because we have skewed dataset. Most image pairs are of different people. Hence, showing the same people is the minority class. For this reason, it is important to check the recall and precision. As you can see, the recall rate is higher than precision for threshold 0.7 by about 30%. And for threshold value 0.8, 0.85 and 0.9, the precision is higher than the recall, and the difference between recall and precision increases when the threshold value increases, and precision is higher. This means for these thresholds, the classifier lean to either positive or negative side. For threshold value 0.75, there is a slight difference between recall and precision, which shows a good balance between recall and precision when they are both higher than 0.7. That's why I choose 0.75 as the threshold to use on the testing dataset.

Threshold 0.7
Training accuracy is:0.9128430716266616
Training recall is:0.816153546375682
Training precision is:0.536849525762625
Confusion matrix 
 [8377 7227 ]
[1887 87079]

Validation accuracy is:0.93
Validation recall is:0.8488372093023255
Validation precision is0.5615384615384615
Confusion matrix 
 [73 57 ]
[13 857]


Threshold 0.75
Training accuracy is:0.9444008797934398
Training recall is:0.7422057677318784
Training precision is:0.7062859262006305
Confusion matrix 
 [7618 3168 ]
[2646 91138]

Validation accuracy is:0.958
Validation recall is:0.7906976744186046
Validation precision is0.7391304347826086
Confusion matrix 
 [68 24 ]
[18 890]


Threshold 0.8
Training accuracy is:0.9549297121545376
Training recall is:0.6457521434138738
Training precision is:0.8602206359506813
Confusion matrix 
 [6628 1077 ]
[3636 93229]

Validation accuracy is:0.969
Validation recall is:0.7209302325581395
Validation precision is0.8985507246376812
Confusion matrix 
 [62 7 ]
[24 907]


Threshold 0.85
Training accuracy is:0.9511714640910395
Training recall is:0.5222135619641465
Training precision is:0.9636821287306724
Confusion matrix 
 [5360 202 ]
[4904 94104]

Validation accuracy is:0.962
Validation recall is:0.5813953488372093
Validation precision is0.9615384615384616
Confusion matrix 
 [50 2 ]
[36 912]


Threshold 0.9
Training accuracy is:0.9382231997704886
Training recall is:0.3706157443491816
Training precision is:1.0
Confusion matrix 
 [3804 0 ]
[6460 94306]

Validation accuracy is:0.944
Validation recall is:0.3488372093023256
Validation precision is1.0
Confusion matrix 
 [30 0 ]
[56 914]


[1] J. P. Lewis, “Fast Normalized Cross-Correlation”, Industrial Light and Magic.

In [7]:
def template_matching():
    #Repeatedly get the two images in a pair, and calculate the similarity score
    threshold=0.75

    #For training dataset    
    correct_predicted_same = 0 #TP
    correct_predicted_diff = 0 #TN
    wrongly_predicted_same = 0 #FP
    wrongly_predicted_diff = 0 #FN
    ##For 1.3, combining images
    #train_im_combined=[]
    #train_lb_combined=[]
    for num in range(0,104570):#104570 for training set
        if (num%1000 == 1):
            print(num)
        i=train_i_list[num]
        j=train_j_list[num]
    
        co_im1=train_images[i]
        lb_im1=train_labels[i]
    
        co_im2=train_images[j]
        lb_im2=train_labels[j]   
    
        result = match_template(co_im1, co_im2)

        if result>threshold:
            if lb_im1 == lb_im2:
                correct_predicted_same=correct_predicted_same+1
            else:
                wrongly_predicted_same=wrongly_predicted_same+1
        else:
            if lb_im1 != lb_im2:
                correct_predicted_diff=correct_predicted_diff+1
            else:
                wrongly_predicted_diff=wrongly_predicted_diff+1
            
    print(correct_predicted_same)
    print(correct_predicted_diff)
    print(wrongly_predicted_same)
    print(wrongly_predicted_diff)
    accuracy=(correct_predicted_same+correct_predicted_diff)/(correct_predicted_same+correct_predicted_diff+wrongly_predicted_same+wrongly_predicted_diff)
    recall=(correct_predicted_same)/(correct_predicted_same+wrongly_predicted_diff) #Assume the same pair is the positive
    precision=(correct_predicted_same)/(correct_predicted_same+wrongly_predicted_same)
    print("Training accuracy is:"+str(accuracy))
    print("Training recall is:"+str(recall))
    print("Training precision is:"+str(precision))
    print("Confusion matrix \n ["+str(correct_predicted_same)+" "+str(wrongly_predicted_same)+" ]\n"+"["+str(wrongly_predicted_diff)+" "+str(correct_predicted_diff)+"]")

    
    #For validation dataset
    correct_predicted_same = 0 #TP
    correct_predicted_diff = 0 #TN
    wrongly_predicted_same = 0 #FP
    wrongly_predicted_diff = 0 #FN

    for num in range(0,1000):#1000 for validation set
        i=valid_i_list[num]
        j=valid_j_list[num]
    
        co_im1=train_images[i]
        lb_im1=train_labels[i]
    
        co_im2=train_images[j]
        lb_im2=train_labels[j]
        
        result = match_template(co_im1, co_im2)
        if result>threshold:
            if lb_im1 == lb_im2:
                correct_predicted_same=correct_predicted_same+1
            else:
                wrongly_predicted_same=wrongly_predicted_same+1
        else:
            if lb_im1 != lb_im2:
                correct_predicted_diff=correct_predicted_diff+1
            else:
                wrongly_predicted_diff=wrongly_predicted_diff+1

    print(correct_predicted_same)
    print(correct_predicted_diff)
    print(wrongly_predicted_same)
    print(wrongly_predicted_diff)
    accuracy=(correct_predicted_same+correct_predicted_diff)/(correct_predicted_same+correct_predicted_diff+wrongly_predicted_same+wrongly_predicted_diff)
    recall=(correct_predicted_same)/(correct_predicted_same+wrongly_predicted_diff) #Assume the same pair is the positive
    precision=(correct_predicted_same)/(correct_predicted_same+wrongly_predicted_same)
    print("Validation accuracy is:"+str(accuracy))
    print("Validation recall is:"+str(recall))
    print("Validation precision is"+str(precision))
    print("Confusion matrix \n ["+str(correct_predicted_same)+" "+str(wrongly_predicted_same)+" ]\n"+"["+str(wrongly_predicted_diff)+" "+str(correct_predicted_diff)+"]")

template_matching()


1
1001
2001
3001
4001
5001
6001
7001
8001
9001
10001
11001
12001
13001
14001
15001
16001
17001
18001
19001
20001
21001
22001
23001
24001
25001
26001
27001
28001
29001
30001
31001
32001
33001
34001
35001
36001
37001
38001
39001
40001
41001
42001
43001
44001
45001
46001
47001
48001
49001
50001
51001
52001
53001
54001
55001
56001
57001
58001
59001
60001
61001
62001
63001
64001
65001
66001
67001
68001
69001
70001
71001
72001
73001
74001
75001
76001
77001
78001
79001
80001
81001
82001
83001
84001
85001
86001
87001
88001
89001
90001
91001
92001
93001
94001
95001
96001
97001
98001
99001
100001
101001
102001
103001
104001
8377
87079
7227
1887
Training accuracy is:0.9128430716266616
Training recall is:0.816153546375682
Training precision is:0.536849525762625
Confusion matrix 
 [8377 7227 ]
[1887 87079]
73
857
57
13
Validation accuracy is:0.93
Validation recall is:0.8488372093023255
Validation precision is0.5615384615384615
Confusion matrix 
 [73 57 ]
[13 857]


1.3 Perceptron
First of all, I concatenated each image pairs into an image of 128 by 64. Meanwhile I label the concatenated images as 0 when they are the same person, as 1 when they are different persons.

In [4]:
train_im_combined=[]
train_lb_combined=[]
for num in range(0,104570):#104570 for training set
    i=train_i_list[num]
    j=train_j_list[num]
    
    co_im1=train_images[i]
    lb_im1=train_labels[i]

    co_im2=train_images[j]
    lb_im2=train_labels[j]
    #Concatenate Images and label them  
    combined_image = np.concatenate((co_im1,co_im2),axis=1)
    combined_image_f=combined_image.flatten()
    train_im_combined.append(combined_image_f)
    if lb_im1 == lb_im2:
        train_lb_combined.append(0);
    else:
        train_lb_combined.append(1);
print(len(train_im_combined))
print(len(train_lb_combined))

#For validation dataset
##For 1.3, combining images
valid_im_combined=[]
valid_lb_combined=[]
for num in range(0,1000):#1000 for validation set
    i=valid_i_list[num]
    j=valid_j_list[num]
    
    co_im1=train_images[i]
    lb_im1=train_labels[i]
    
    co_im2=train_images[j]
    lb_im2=train_labels[j]
    #Concatenate Images and label them  
    combined_image = np.concatenate((co_im1,co_im2),axis=1)
    combined_image_f=combined_image.flatten()
    valid_im_combined.append(combined_image_f)
    if lb_im1 == lb_im2:
        valid_lb_combined.append(0);
    else:
        valid_lb_combined.append(1);
print(len(valid_im_combined))
print(len(valid_lb_combined))

test_im_combined = []
test_lb_combined = []
for num in range(0, 73536):  # 73536 for testing set
    i = test_i_list[num]
    j = test_j_list[num]

    co_im1 = test_images[i]
    lb_im1 = test_labels[i]

    co_im2 = test_images[j]
    lb_im2 = test_labels[j]
    # Concatenate Images and label them
    combined_image = np.concatenate((co_im1, co_im2), axis=1)
    combined_image_f = combined_image.flatten()
    test_im_combined.append(combined_image_f)
    if lb_im1 == lb_im2:
        test_lb_combined.append(0);
    else:
        test_lb_combined.append(1);
print(len(test_im_combined))
print(len(test_lb_combined))

train_im_combined=np.array(train_im_combined)
train_lb_combined=np.array(train_lb_combined)
valid_im_combined=np.array(valid_im_combined)
valid_lb_combined=np.array(valid_lb_combined)
test_im_combined=np.array(test_im_combined)
test_lb_combined=np.array(test_lb_combined)


104570
104570
1000
1000
73536
73536


I built and trained a multi-layer perceptron using Keras in Python. Initially I intended to use MLP function in sklearn. but my laptop cannot run it due to memory error, because of the limit of my laptop RAM. Then I switched to Keras and it worked.

I fit the training data (104570) into the model, and use validation data (1000) to get accuracy, recall, precision and confusion matrix to review and modify the network architecture. Finally I decided my model has two hidden layers, each of them has 50 neurons. This is because this model perform best among all the models I tried, in terms of precision and recall mainly. This is because our data is imblanced, and label 0 is the minority class so that accuracy is not that useful here. Some other models I tried are listed below for comparison.
As you can see, the 50 by 50 model is the best. The precison and recall are all very high. And the confusion matrix is the best among the followings.
And have a look at confusion matrix, you will find that for 1 layer with 30 neurons there is no true positives, which means the model simply predict all the input into being different persons, which is the majority class. This is because most of our data samples are being different persons, so that the classifier just predict as the majority which is wrong. I think this is because the model is too simple. That's why I increase the number of layers to two, and three as you can see.

The activation layers are Rectified Linear Unit for hidden layers. For output layer, the activation function is sigmoid because this is a binary classification.

Optimizer is Adam Optimizer, which is most popular now.

I changed loss function to binary_crossentropy once, and I changed epochs to 2 and 5. However, these changes do not make a visible change on the output.


2 hidden layers, 50 by 50 neurons
104570/104570 [==============================] - 176s 2ms/step - loss: 0.0669 - accuracy: 0.9151
Valid accuracy: 98.1000000000%
Valid precision:98.7709497207%
Valid recall:99.1031390135%
Valid confusion matrix is:
[[ 97  11]
 [  8 884]]


1 hidden layer with 30 neurons
Valid accuracy: 90.5000000000%
Valid precision:90.5000000000%
Valid recall:100.0000000000%
Valid confusion matrix is:
[[  0  95]
 [  0 905]]


2 hidden layers, 30 by 10 neurons
Valid accuracy: 90.9000000000%
Valid precision:91.2032355915%
Valid recall:99.5584988962%
Valid confusion matrix is:
[[  7  87]
 [  4 902]]

3 hidden layesr, 90 by 60 by 30
Valid accuracy: 92.6000000000%
Valid precision:97.8235967927%
Valid recall:93.9493949395%
Valid confusion matrix is:
[[ 72  19]
 [ 55 854]]

In [5]:
from keras.models import Sequential
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt

mlp = Sequential()

from keras.layers import Dense
mlp.add(Dense(units=50, input_shape=(8192,), activation='relu')) #8192 is the nodes in the input layer, which is the number of pixels 128*64
mlp.add(Dense(units=50, activation='relu'))
mlp.add(Dense(units=1, activation='sigmoid')) #output layer

mlp.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
mlp.fit(train_im_combined, train_lb_combined, epochs=1, batch_size=32)
print("model fitted")

Using TensorFlow backend.


Epoch 1/1
104570/104570 [==============================] - 159s 2ms/step - loss: 0.0690 - accuracy: 0.9112
model fitted


In [9]:
valid_pdlb_combined = mlp.predict_classes(valid_im_combined)
#print(valid_pdlb_combined)
valid_accuracy = accuracy_score(valid_lb_combined, valid_pdlb_combined)
print("Valid accuracy: {:.10f}%".format(valid_accuracy * 100))
valid_precision = precision_score(valid_lb_combined, valid_pdlb_combined)
print("Valid precision:{:.10f}%".format(valid_precision * 100))
valid_recall = recall_score(valid_lb_combined, valid_pdlb_combined)
print("Valid recall:{:.10f}%".format(valid_recall * 100))
valid_confu_matrix = confusion_matrix(valid_lb_combined,valid_pdlb_combined)
print("Valid confusion matrix is:")
print(valid_confu_matrix)

Valid accuracy: 93.6000000000%
Valid precision:93.4426229508%
Valid recall:100.0000000000%
Valid confusion matrix is:
[[ 24  64]
 [  0 912]]
1000/1000 [==============================] - 0s 184us/step
[0.037875514088198545, 0.9359999895095825]


1.4 Classification Comparison
I use threshold 0.75 for template matching model, and use 50 by 50 hidden layers for MLP model. The result of testing are shown below.

For Template Matching:
Testing accuracy is: 91.58235422106179% (0.9158235422106179)
Testing recall is: 80.25568181818182% (0.8025568181818182)
Testing precision is 59.98938241019288% (0.5998938241019288)
Confusion matrix 
 [6780 4522 ]
[1668 60566]

For MLP:
Test accuracy: 81.6430047868%
Test recall:83.5345993117%
Test precision:95.1323640054%
Test confusion matrix is:
[[ 5666  2782]
 [10717 54371]]

If we look at accuracy, Template Matching has an accuracy higher than MLP by about 10%. But accuracy may not be a good indicator for performance because in the dataset, there are much more negatives than positives. If the classifier simply predict all the testing data samples as Negative(i.e. different persons), the accuarcy will be (Negative samples/All the samples), which looks really high which is 77.72%=57153/73536. However, the performance is bad apparently.

Therefore, we need to compare recall and precision. Recall is the percentage of true positive sample in all the real positive samples. And precision is the percentage of true positive sample in the predicted positive results. In terms of them, it is obvious that MLP perform better than Template Matching. In particular, the precision of MLP is 36% higher than Template Matching. Therefore, I prefer the Multi-layer Perceptron in terms of performance because of the imbalanced situation of our dataset.

In terms of training effort, the building and the training process for Multi-layer Perceptron model is about 6 mins.The MLP I used has 50*50=2500 connections which requires some computation resources. And the image processing takes a bit more time for Multi-layer Perceptron because we need to concatenate the image pairs. In contrast, Template Matching does not have explicit training process, or model building process. And the image pairs do not need to be concatenated. Therefore, for training effort, I think template matching is better.

About prediction speed, the template matching takes about 6 seconds per 1000 samples. However, the Multi-layer Perceptron takes about 1 second per 1000 samples. MLP is faster than template matching for prediction.

For generalization, I think the MLP would be better because the MLP has a higher complexity compared to template matching. The MLP I used has 50*50=2500 connections in the hidden layers. And based on recall and precision, MLP generalize better for unseen data than template matching.

For robostness, I believe that MLP is better because it is a more complex model than the template matching model. Template matching method compute the cross correlation of two images, which is prone to shift and noise in image. Therefore, MLP will be less prone to the shift and noise of the images because the model constructed is not simply calculating cross correlation between two images.

Last but not the least, in terms of explainability, Template Matching is better because we have no idea what is the meaning of the learned weight and biases in multi-layer perceptron. For template matching, we know that what we got is the cross correlation which shows the similarity of two images, which outputs a correlation coefficient.

In [8]:
#testing for template matching model
def template_matching_testing():
    threshold=0.75
    #For testing dataset
    correct_predicted_same = 0 #TP
    correct_predicted_diff = 0 #TN
    wrongly_predicted_same = 0 #FP
    wrongly_predicted_diff = 0 #FN

    for num in range(0,73536):#73536 for testing set
        i=test_i_list[num]
        j=test_j_list[num]
    
        co_im1=test_images[i]
        lb_im1=test_labels[i]
    
        co_im2=test_images[j]
        lb_im2=test_labels[j]
    
        result = match_template(co_im1, co_im2)
        if result>threshold:
            if lb_im1 == lb_im2:
                correct_predicted_same=correct_predicted_same+1
            else:
                wrongly_predicted_same=wrongly_predicted_same+1
        else:
            if lb_im1 != lb_im2:
                correct_predicted_diff=correct_predicted_diff+1
            else:
                wrongly_predicted_diff=wrongly_predicted_diff+1

    print(correct_predicted_same)
    print(correct_predicted_diff)
    print(wrongly_predicted_same)
    print(wrongly_predicted_diff)
    accuracy=(correct_predicted_same+correct_predicted_diff)/(correct_predicted_same+correct_predicted_diff+wrongly_predicted_same+wrongly_predicted_diff)
    recall=(correct_predicted_same)/(correct_predicted_same+wrongly_predicted_diff) #Assume the same pair is the positive
    precision=(correct_predicted_same)/(correct_predicted_same+wrongly_predicted_same)
    print("Testing accuracy is:"+str(accuracy))
    print("Testing recall is:"+str(recall))
    print("Testing precision is"+str(precision))
    print("Confusion matrix \n ["+str(correct_predicted_same)+" "+str(wrongly_predicted_same)+" ]\n"+"["+str(wrongly_predicted_diff)+" "+str(correct_predicted_diff)+"]")

template_matching_testing()

6780
60566
4522
1668
Testing accuracy is:0.9158235422106179
Testing recall is:0.8025568181818182
Testing precision is0.5998938241019288
Confusion matrix 
 [6780 4522 ]
[1668 60566]


In [10]:
#testing for MLP model

test_pdlb_combined = mlp.predict_classes(test_im_combined)
print(test_pdlb_combined)
test_accuracy = accuracy_score(test_lb_combined, test_pdlb_combined)
print("Test accuracy: {:.10f}%".format(test_accuracy * 100))
test_precision = precision_score(test_lb_combined, test_pdlb_combined)
print("Test precision:{:.10f}%".format(test_precision * 100))
test_recall = recall_score(test_lb_combined, test_pdlb_combined)
print("Test recall:{:.10f}%".format(test_recall * 100))
test_confu_matrix = confusion_matrix(test_lb_combined,test_pdlb_combined)
print("Test confusion matrix is:")
print(test_confu_matrix)

[[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]
Test accuracy: 88.6151000870%
Test precision:88.6654304491%
Test recall:99.9093534907%
Test confusion matrix is:
[[  135  8313]
 [   59 65029]]


NameError: name 'model' is not defined